In [1]:
!pip3 install vncorenlp
import numpy as np
# %env DATA_DIR=./data/squad 
# !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
# !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!pip install transformers==3.5.1
!pip install torch==1.4.0
# !pip install sentencepiece

     |████████████████████████████████| 2.6 MB 6.8 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=4a1248df8b44a04ef4284207ee75e807b54f6ba7db4275f348f99732e8c73a44
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
     |████████████████████████████████| 1.3 MB 6.9 MB/s 
     |████████████████████████████████| 2.9 MB 46.8 MB/s 
     |████████████████████████████████| 1.1 MB 51.3 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0

In [2]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (215/215), 214.22 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_datasets.question_answering import xquad
import json
import numpy as np

In [4]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [5]:
dataset = xquad.Xquad(config='vi')

In [6]:
dataset.download_and_prepare()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/xquad/vi/2.0.0.incompleteKQGZM2/xquad-test.tfrecord


...

  0%|          | 0/10555 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/xquad/vi/2.0.0.incompleteKQGZM2/xquad-translate-test.tfrecord


  0%|          | 0/1178 [00:00<?, ? examples/s]

Dataset xquad downloaded and prepared to /root/tensorflow_datasets/xquad/vi/2.0.0. Subsequent calls will reuse this data.


In [33]:
ds = dataset.as_dataset(split='translate-train', shuffle_files=True)
# ds = dataset.as_dataset(split='translate-dev', shuffle_files=True)
# ds = dataset.as_dataset(split='test', shuffle_files=True)

INFO:absl:Constructing tf.data.Dataset for split translate-train, from /root/tensorflow_datasets/xquad/vi/2.0.0


In [34]:
df = tfds.as_dataframe(ds, dataset.info)

In [35]:
#For training sample
df = df.applymap(lambda x: x.decode('utf-8', 'ignore') if isinstance(x, bytes) else x)
df = df.applymap(lambda x: x[0].decode('utf-8', 'ignore') if isinstance(x, np.ndarray) and isinstance(x[0], bytes) else x)

In [36]:
# # For dev/test sample
# df = df.applymap(lambda x: x.decode('utf-8', 'ignore') if isinstance(x, bytes) else x)
# df = df.applymap(lambda x: [x_i.decode('utf-8', 'ignore') for x_i in x] if isinstance(x, np.ndarray) and isinstance(x[0], bytes) else x)

In [37]:
df.head(5)

,answers/answer_start,answers/text,context,id,question,title
0,[212],12 tháng 11 năm 1962,Phiên họp đầu tiên kết thúc trong một buổi lễ ...,572805304b864d1900164252,Khi nào phiên tiếp theo được công bố?,
1,[116],chúng vẫn ấm và khô qua mùa đông,Bewick sau đó mô tả một thí nghiệm đã thành cô...,5705f3a375f01819005e77c4,Làm thế nào mà thí nghiệm thành công?,
2,[169],Zdzisław Jachimecki,Thành công của Chopin với tư cách là nhà soạn ...,56cf6ca44df3c31400b0d777,"Ai nói rằng Chopin đặt ra ""vào thế giới rộng l...",
3,[136],Thống đốc Chiết Giang,Chính trị của Chiết Giang được cấu trúc trong ...,56f98a729b226e1400dd1541,Ai là quan chức cấp cao nhất trong Chính phủ N...,
4,[12],mười một,"Oklahoma có mười một trường đại học công lập, ...",5727e060ff5b5019007d9754,Oklahoma có bao nhiêu trường đại học công lập?,


In [38]:
def check_loop_string(symbol, string_a, len_s):
  if len(string_a) == 0:
    return len_s
  if symbol == string_a[0]:
    return check_loop_string(symbol, string_a[1:], len_s+1)
  return len_s

In [39]:
special_symbol = ['"',"'", '[',']','(',')','/','\\','{','}','&', '*', '?', ';', ':', '.', ',', '%', '!', '#']
def refine_senc(ori_sen,list_sens):
  senc = [word for sen in list_sens for word in sen]
  for idx, word in enumerate(senc):
    if word[0] == "_" and len(word) > 1:
      senc[idx] = word[1:]
    for symbol in special_symbol:
      if senc[idx].find(symbol) != 0 and senc[idx].find(symbol) != -1 or (senc[idx].find(symbol) == 0 and len(senc[idx])!=0) or (senc[idx].find("_") == 1 and len(senc[idx])==3):
        senc[idx] = senc[idx].replace("_", "")
  senc = "".join(senc)
  for idx, char in enumerate(ori_sen):
    if (char == " " and senc[idx] != "_") or (char == " " and (check_loop_string("_", senc[idx:], 0)>1)):
      senc = senc[:idx] + ' ' + senc[idx:]
  return senc

In [40]:
import transformers
from transformers.data.processors.squad import SquadV1Processor
processor = SquadV1Processor()

In [41]:
structure_squadv1 = {
    'version' : '1.1',
    'data' : []
}

In [42]:
from tqdm import tqdm
import pandas as pd

In [44]:
for idx, row in tqdm(df.iterrows()):
  # answer_text = [refine_senc(row['answers/text'][i], annotator.tokenize(row['answers/text'][i])) for i in range(len(row['answers/answer_start']))]
  answer_text = [refine_senc(row['answers/text'], annotator.tokenize(row['answers/text']))]
  context = refine_senc(row['context'], annotator.tokenize(row['context']))
  sample = {
    'title': row['title'],
    'paragraphs': [
    {
        'context': context,
        'qas': [{ 'id': row['id'],
                  'question': refine_senc(row['question'], annotator.tokenize(row['question'])),
                  'answers': [{ 'answer_start': int(row['answers/answer_start'][i]),
                                'text': context[row['answers/answer_start'][i]:row['answers/answer_start'][i]+len(answer_text[i])] } for i in range(len(row['answers/answer_start']))] 
                }]
    }]}
  structure_squadv1['data'].append(sample)

87187it [22:00, 66.03it/s]


In [49]:
  structure_squadv1['data'][1000]

{'paragraphs': [{'context': 'Các tính_năng bảo_mật mới trong Windows 8 bao_gồm hai phương_thức xác_thực mới được điều_chỉnh theo màn_hình cảm_ứng (PIN và mật_khẩu hình_ảnh), bổ_sung các khả_năng chống vi-rút cho Windows_Defender (đưa nó ngang_hàng với Microsoft Security_Essentials). Lọc_SmartScreen được tích_hợp vào Windows, Family_Safety cung_cấp các điều_khiển của Cha_mẹ, cho_phép cha_mẹ giám_sát và quản_lý các hoạt_động của con_cái họ trên một thiết_bị có báo_cáo hoạt_động và kiểm_soát an_toàn. Windows 8 cũng cung_cấp phục_hồi hệ_thống tích_hợp thông_qua các chức_năng "Làm mới" và "Đặt lại" mới, bao_gồm khôi_phục hệ_thống từ ổ USB. Bản vá bảo_mật đầu_tiên của Windows 8 sẽ được phát_hành vào ngày 13 tháng 11 năm 2012; nó sẽ chứa ba bản sửa lỗi được coi là "quan_trọng" của công_ty.',
   'qas': [{'answers': [{'answer_start': 118,
       'text': 'PIN và mật_khẩu hình_ảnh'}],
     'id': '57302b2ab2c2fd14005689c7',
     'question': 'Windows 8 cung_cấp những tính_năng bảo_mật mới nào?'}]}]

In [50]:
df.iloc[1000]

answers/answer_start                                                [118]
answers/text                                     PIN và mật khẩu hình ảnh
context                 Các tính năng bảo mật mới trong Windows 8 bao ...
id                                               57302b2ab2c2fd14005689c7
question                Windows 8 cung cấp những tính năng bảo mật mới...
title                                                                    
Name: 1000, dtype: object

In [ ]:
with open('./train_xquad_ws.json', 'w') as outfile:
    json.dump(structure_squadv1, outfile)